In [150]:
from __future__ import print_function

import torch
import torch.optim as optim

from torch.utils.data.dataset import Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
torch.backends.cudnn.bencmark = True
import torchvision
import torchvision.transforms as transforms

import os,sys,cv2,random,datetime,time,math
import argparse
import numpy as np

from net_s3fd import *
#from s3fd_gender import *
#import s3fd_gender
from s3fd import *
import s3fd
from bbox import *
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image

In [151]:
class CelebDataset(Dataset):
    """Dataset wrapping images and target labels
    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['Image_Name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['Image_Name']
        #gender training
        self.y_train = self.mlb.fit_transform(tmp_df['Gender'].str.split()).astype(np.float32)
     
    #resizing the input image
    def __getitem__(self, index):
        img = cv2.imread(self.img_path + self.X_train[index] + self.img_ext)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        #img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        #img = Variable(torch.from_numpy(img).float(),volatile=True)
        
        #if self.transform is not None:
        #    img = self.transform(img)
        
        #label === gender 
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [152]:
transformations = transforms.Compose(
    [
     transforms.ToTensor()
     
     #transforms.Normalize(mean=[104,117,123])
     ])

In [153]:
train_data = "index.csv"
img_path = "data/Celeb_Small_Dataset/"
img_ext = ".jpg"
dset = CelebDataset(train_data,img_path,img_ext,transformations)
train_loader = DataLoader(dset,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

In [154]:
def save(model, optimizer, loss, filename):
    save_dict = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.data[0]
        }
    torch.save(save_dict, filename)

In [175]:
def train_model(model, criterion, optimizer, num_classes, num_epochs = 100):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0

        for i,(img,label) in enumerate(train_loader):
            img = img.view((1,)+img.shape[1:])
            genlist = []
            #loss = torch.tensor(1,2)
            #loss = []
                        
            #if use_cuda:
            #    data, target = Variable(img.cuda()), Variable(torch.Tensor(label).cuda())
            #else:
            data, target = Variable(img), Variable(torch.Tensor(label))
            target = target.view(num_classes,1)
            
            #output as desired form
            
            
            #apply soft max to gender layers

            optimizer.zero_grad()
            #run through model
            outputs = model(data)
            #print(outputs)
                
            #apply softmax to layers
            for j in range(len(outputs)//2): outputs[j*2] = F.softmax(outputs[j*2])
            #fetch data
            for j in range(len(outputs)//2): 
                ocls,ogen = outputs[i*2].data.cpu(),outputs[i*2+1].data.cpu()
                #print(ocls)
                
                FB,FC,FH,FW = ocls.size() # feature map size
                print(FH, FW)
                stride = 2**(i+2)    # 4,8,16,32,64,128
                anchor = stride*4
                for Findex in range(FH*FW):
                    windex,hindex = Findex%FW,Findex//FW
                    axc,ayc = stride/2+windex*stride,stride/2+hindex*stride
                    score = ocls[0,1,hindex,windex]
                    #loc = oreg[0,:,hindex,windex].contiguous().view(1,4)
                    if score<0.05: continue
                    gen = ogen[0,:,hindex,windex].contiguous().view(1,2)
                    print(gen)
                    #Append this gender prediction scores (1x2 dimension) to a list.
                    #genlist.append(gen)
                    #loss += criterion(gen,target)
                    genlist.append(gen)
                  
            #For each of the predictions in gender prediction list: 
            #calculate the loss and add it to the total loss for this iteration.
            
            for k,(gen) in enumerate(genlist):
                loss += criterion(gen, target)
                
            if i%50==0:
                print("Reached iteration ",i)
                running_loss += loss.data[0]
                #running_loss += loss
            
            #Update
            loss.backward()
            optimizer.step()
            
            #total running loss
            running_loss += loss.data[0]
            #running_loss += loss
            
        if epoch % 10 == 0:
            save(model, optimizer, loss, 'faceRecog.saved.model')
        print(running_loss)

In [176]:
#combine our model with the pre-trained model

num_classes = 2
myModel = (s3fd.s3fd_original(num_classes))
loadedModel = torch.load('s3fd_convert.pth')

newModel = myModel.state_dict()
pretrained_dict = {k: v for k, v in loadedModel.items() if k in newModel}

new_params = [k for k in newModel if k not in loadedModel]

newModel.update(pretrained_dict)
myModel.load_state_dict(newModel)

In [177]:
#if using GPU
use_cuda = True
myModel.eval()

s3fd_original(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d

In [178]:
temp1 = myModel.conv4_3_norm_mbox_conf.weight[0][None,:,:,:]
temp2 = myModel.conv5_3_norm_mbox_conf.weight[0][None,:,:,:]
temp3 = myModel.fc7_mbox_conf.weight[0][None,:,:,:]
temp4 = myModel.conv6_2_mbox_conf.weight[0][None,:,:,:]

temp1 = torch.cat((temp1,temp1),0)
temp2 = torch.cat((temp2,temp2),0)
temp3 = torch.cat((temp3,temp3),0)
temp4 = torch.cat((temp4,temp4),0)
temp3.size()

torch.Size([2, 1024, 3, 3])

In [179]:
criterion = nn.MSELoss()
#criterion = nn.BCEloss()


# Turn the requires_grad off for all the parameters in your model.
# except the fc layers, ****************
#for param in myModel.parameters():
#    if param in myModel.fc7.parameters():
#        param.requires_grad = True 
#    else:
#        param.requires_grad = False

for pair in myModel.named_parameters():
    if pair[0] not in new_params:
        pair[1].requires_grad = False
        
        
# myModel.conv4_3_norm_mbox_conf.weight[0] += genTensor4_3
# Add this new tensor to the Gender layer's weight. 
# Make sure it has dimension N x C x H x W = N x 2 x H x W. (N is the number of images in the batch)


#Stochastic Gradient Descent.
#optimizer = optim.SGD(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(filter(lambda p: p.requires_grad,myModel.parameters()), lr=0.0001, betas=(0.9, 0.999))

#if use_cuda: 
#    myModel = myModel.cuda()

# Call the training function defined above.    
model_ft = train_model(myModel, criterion, optimizer, num_classes, num_epochs=100)

Epoch 0/99
----------
f3_3
torch.Size([1, 256, 32, 32])
f4_3
torch.Size([1, 512, 16, 16])
f5_3
torch.Size([1, 512, 8, 8])
ffc7
torch.Size([1, 1024, 12, 12])
f6_2
torch.Size([1, 512, 6, 6])
f7
torch.Size([1, 256, 3, 3])
resized_cls1
torch.Size([1, 2, 64, 64])
32 32
32 32
32 32
32 32
Reached iteration  0


//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


UnboundLocalError: local variable 'loss' referenced before assignment

In [ ]:
#Test the input and output

def transform(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (256,256))
        img = img - np.array([104,117,123])
        img = img.transpose(2, 0, 1)
        
        img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).float()
        
        return Variable(img.cuda())

    
myModel = myModel.cuda()
testImage1 = transform('data/Test/TestCeleb_4/25-FaceId-0.jpg')
testImage2 = transform('data/Test/TestCeleb_4/26-FaceId-0.jpg')
testImage3 = transform('data/Test/TestCeleb_4/27-FaceId-0.jpg')
testImage4 = transform('data/Test/TestCeleb_10/25-FaceId-0.jpg')
testImage5 = transform('data/Test/TestCeleb_10/26-FaceId-0.jpg')
testImage6 = transform('data/Test/TestCeleb_10/24-FaceId-0.jpg')

output1 = myModel(testImage1)
output2 = myModel(testImage2)
output3 = myModel(testImage2)
output4 = myModel(testImage4)
output5 = myModel(testImage5)
output6 = myModel(testImage6)
print("testImage1 - ",output1)
print("testImage2 - ",output2)
print("testImage3 - ",output3)
print("testImage1 - ",output4)
print("testImage2 - ",output5)
print("testImage3 - ",output6)